# **Project Hint - Reading the Data from Database**

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Step 1 - Reading the Tables from Database file**

In [ ]:
# Read the code below and write your observation in the next cell

conn = sqlite3.connect('/content/drive/My Drive/eng_subtitles_database.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('zipfiles',)]


**In the above cell, I am able to read the table inside the database. As mentioned earlier, table name is `zipfiles`. We also know from README.txt that this table contains three columns: 'num', 'name' and 'content'.**

## **Step 2 - Reading the columns of Table**

In [ ]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

num
name
content


**The above code helps in checking the column names in the database table.**

**Let's now use `SELECT * FROM zipfiles` to read all the data into a `df` variable.**

## **Step 3 - Loading the Database Table inside a Pandas DataFrame**

In [ ]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


**Looks like the `content` column donot contain the subtitles text. Instead as mentioned in README.txt, it might be latin-1 encoded.**

## **Step 4 - Printing `content` of 0th Row**

In [ ]:
b_data = df.iloc[0, 2]

# here 2 represent the index of content column
# 0 represents the row number

In [ ]:
print(b_data)

b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x99V\x9fx\x96\xf0\x8c\x9e\x00\x00\x86\x9b\x01\x00;\x00\x00\x00The.Message.1976.REMASTERED.1080p.BluRay.x264-PiGNUS.EN.srt\xad\xbdm\x93\xdc\xc6\x91.\xfa\x9d\x11\xfc\x0f-}\xe1=\x11-\x9d\x06P\x85\x17\x9d\x8d\xd5%%[\xa4-Y>&u\x15>\xdf\xd0\xd3\x98\x19x\xfae\x0cts<\xfe\xf57\x9f\'\xb3\n\xd9\xa4\xbc\xbb\xf7\xc6Fl\xacELW\xa2\xaa\x90\x95\x95\xafO\x16/_l6\xdf\xe0\xff\xea\xf5f\xb3Y}\xf5\xd5\xbf\xaf\xf4AQ\xae7Mx\xf9\xe2\xd7\xfe|s\xbf\xea\x8f\xcf\xab\x8f\xe3n8\xadN\xc7\xfdx\x1cVO\xe3\xf9~\xf5\xf3\xe3p\xfc\xea\xfd/o>\xbc\xfb\xf0\xe3\xef\xde\xbf|\xf1\xfbi\x18Vo\xa6\xd3\xd3<L\xab\xe1\x1f\xe7\xe18\x8f\xa7\xe37\xab\xd3\xbc\xdb~-\xc3\x1e\xfe\xa7<|\xf9\xe2\xe5\x8bR_[~S\xd6\xeb\xa2k\xf3k\xe5A\xb7\xeeb\xf5\xf2\xc5\xbb\xe3\xea|?\xac\x8e\xfdaX\x9dnW?\x9cvk>8\x9c\xe6\xf3\xean\xeao\xc6\xd3ev\x8f~\x1a\xa6\x9b\xf1\xf6\xb2\xff\x1a\xe4\xabD\xbe*d\x11\xa5#_U\xeb\xaa\xd9`\xa6\xa7\xc3\xea\xa7\xcb}\x7f8\xf4F\xf9\xa7a\x9e\x87\xe3\x9d\xcc\\\xdf\x07B!\x13\xaa\xd61n<!\xd9\xaf\xd0\

**From the content, it appears to start with the bytes "PK\x03\......", which suggests that it might be a ZIP archive file. How do I know it? Experience! I have worked with something similar earlier.**

## **Step 5 - Unzipping the content of 385th row and decoding using `latin-1`**

In [ ]:
import zipfile
import io

# Assuming 'content' is the binary data from your database
binary_data = df.iloc[385, 2]

# Decompress the binary data using the zipfile module
with io.BytesIO(binary_data) as f:
    with zipfile.ZipFile(f, 'r') as zip_file:
        # Reading only one file in the ZIP archive
        subtitle_content = zip_file.read(zip_file.namelist()[0])

# Now 'subtitle_content' should contain the extracted subtitle content
print(subtitle_content.decode('latin-1'))  # Assuming the content is latin-1 encoded text

1
00:00:06,000 --> 00:00:12,074
Watch any video online with Open-SUBTITLES
Free Browser extension: osdb.link/ext

2
00:00:15,370 --> 00:00:16,506
You lose everything, my girl.

3
00:00:16,530 --> 00:00:19,360
So you've said - four times.

4
00:00:20,330 --> 00:00:22,120
I definitely had
it on yesterday.

5
00:00:22,465 --> 00:00:25,785
Your gloves, your keys, that
handkerchief I embroidered for you

6
00:00:25,809 --> 00:00:26,168
Everything!

7
00:00:26,192 --> 00:00:27,280
Five times.

8
00:00:31,610 --> 00:00:32,920
Miss Scarlet?
- Yes.

9
00:00:36,390 --> 00:00:37,390
I'm Miss Scarlet.

10
00:00:37,872 --> 00:00:40,880
May I inquire if
you've lost something?

11
00:00:41,350 --> 00:00:42,530
Some jewellery perhaps?

12
00:00:42,870 --> 00:00:45,130
Yes, my mother's wedding ring.

13
00:00:45,220 --> 00:00:45,840
Have you found it?

14
00:00:45,950 --> 00:00:47,656
Does your ring have
an inscription?

15
00:00:48,650 --> 00:00:51,720
From my father to my mother 'For
my beloved, Livi

**Look's like it worked.**

## **Step 6 - Applying the above Function on the Entire Data**

In [ ]:
import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])

    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [ ]:
df['file_content'] = df['content'].apply(decode_method)

df.head()

,num,name,content,file_content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   num           82498 non-null  int64 
 1   name          82498 non-null  object
 2   content       82498 non-null  object
 3   file_content  82498 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.5+ MB


In [ ]:
df.tail()

,num,name,content,file_content
82493,9521935,the.prophets.game.(2000).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xb8\xa6\x...,"ï»¿1\r\n00:01:16,284 --> 00:01:19,537\r\nGod,\..."
82494,9521937,west.beirut.(1998).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x13\x97\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\napi.Open..."
82495,9521938,frankenstein.the.true.story.(1973).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00$\x97\x9aV...,"1\r\n00:00:01,001 --> 00:00:04,630\r\n(Dramati..."
82496,9521940,frankenstein.the.true.story.(1973).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x97\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nAdvertis..."
82497,9521941,zombie.island.massacre.(1984).eng.1cd,"b'PK\x03\x04\x14\x00\x00\x00\x08\x00,\x97\x9aV...","1\r\n00:00:01,919 --> 00:00:03,253\r\n(Sharp w..."


In [ ]:
df["file_content"]

,file_content
0,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."
...,...
82493,"ï»¿1\r\n00:01:16,284 --> 00:01:19,537\r\nGod,\..."
82494,"1\r\n00:00:06,000 --> 00:00:12,074\r\napi.Open..."
82495,"1\r\n00:00:01,001 --> 00:00:04,630\r\n(Dramati..."
82496,"1\r\n00:00:06,000 --> 00:00:12,074\r\nAdvertis..."


In [ ]:
df.shape

(82498, 4)

Selecting 30% data

In [ ]:
import pandas as pd


df_sample = df.sample(frac=0.30, random_state=42)

%time print(df_sample.shape)

(24749, 4)
CPU times: user 53 µs, sys: 0 ns, total: 53 µs
Wall time: 57.2 µs


In [ ]:
df_sample = df_sample.drop(columns=["content"])
df_sample.head()

,num,name,file_content
17262,9251120,maybe.this.time.(2014).eng.1cd,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."
7294,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,"1\r\n00:00:09,275 --> 00:00:11,876\r\n¶ Oh, I ..."
47707,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,"1\r\n00:00:07,140 --> 00:00:14,220\r\n<i>Timin..."
29914,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,"1\r\n00:00:06,133 --> 00:00:08,900\r\n[etherea..."
54266,9408707,battlebots.(2015).eng.1cd,"ï»¿1\r\n00:00:01,480 --> 00:00:03,570\r\n[Chri..."


In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
df_sample

,num,name,file_content
17262,9251120,maybe.this.time.(2014).eng.1cd,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."
7294,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,"1\r\n00:00:09,275 --> 00:00:11,876\r\n¶ Oh, I ..."
47707,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,"1\r\n00:00:07,140 --> 00:00:14,220\r\n<i>Timin..."
29914,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,"1\r\n00:00:06,133 --> 00:00:08,900\r\n[etherea..."
54266,9408707,battlebots.(2015).eng.1cd,"ï»¿1\r\n00:00:01,480 --> 00:00:03,570\r\n[Chri..."
...,...,...,...
67460,9458807,kevin.can.wait.s01.e13.ring.worm.(2017).eng.1cd,ï»¿[Script Info]\r\nTitle: Default file\r\nScr...
15296,9244890,bia.s01.e29.episode.1.29.(2019).eng.1cd,"ï»¿1\r\n00:00:03,440 --> 00:00:06,160\r\n-Wher..."
40242,9345965,heroes.s02.e11.chapter.eleven.powerless.(2007)...,"ï»¿1\r\n00:00:01,101 --> 00:00:02,865\r\n<i>Pr..."
56391,9417351,hot.in.cleveland.s05.e09.bad.george.clooney.(2...,"ï»¿1\r\n00:00:01,768 --> 00:00:03,168\r\n<i>- ..."


In [ ]:
import re
import re

def clean_subtitle_name(text):
    # Remove unnecessary parts
    cleaned_text = re.sub(r'\.eng.1cd', '', text)
    # Replace dots with spaces
    cleaned_text = cleaned_text.replace('.', ' ')
    # Capitalize the first letter of each word
    cleaned_text = cleaned_text.title()
    # Convert text to lowercase
    cleaned_text = cleaned_text.lower()
    # Remove redundant spaces
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

# Test the function
df_sample["name"]= df_sample["name"].apply(clean_subtitle_name)
df_sample.head()

,num,name,file_content
17262,9251120,maybe this time (2014),"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."
7294,9211589,down the shore s01 e10 and justice for all (1992),"1\r\n00:00:09,275 --> 00:00:11,876\r\n¶ Oh, I ..."
47707,9380845,uncontrollably fond s01 e07 heartache (2016),"1\r\n00:00:07,140 --> 00:00:14,220\r\n<i>Timin..."
29914,9301436,screen two s13 e04 the precious blood (1996),"1\r\n00:00:06,133 --> 00:00:08,900\r\n[etherea..."
54266,9408707,battlebots (2015),"ï»¿1\r\n00:00:01,480 --> 00:00:03,570\r\n[Chri..."


In [ ]:
def clean_subtitle_content(text):
    # Remove timestamps
    clean_text = re.sub(r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}', '', text)
    # Remove HTML tags
    clean_text = re.sub(r'<[^>]*>', '', clean_text)
    # Remove special characters and symbols except spaces
    clean_text = re.sub(r'[^a-zA-Z0-9\s]', '', clean_text)
    # Remove line numbers and extraneous meta text
    clean_text = re.sub(r"\d+\s*", "", clean_text)
    # Remove newlines
    clean_text = re.sub(r"\r\n", " ", clean_text)
    # Remove special characters (modify as needed)
    clean_text = re.sub(r"[^\w\s.,\-']", "", clean_text)
    clean_text = re.sub(r'\s+', ' ', clean_text)

    # Convert text to lowercase
    clean_text = clean_text.lower()

    return clean_text.strip()

df_sample["file_content"] = df_sample["file_content"].apply(clean_subtitle_content)
df_sample["file_content"].iloc[0]

'watch any video online with opensubtitles free browser extension osdblinkext it couldve been just another summer but as i set foot on the sand that summer suddenly felt different like it was going to be the summer that would change my life the summer of freedom the summer of endless possibilities the summer of ooh aah ooh oh oh oh ooh that was the summer of you and me youre quite the dancer why did you stop come on keep dancing whatever im kidding dont get mad huh what hey im just going to get my towel what stop that you thought i was gonna kiss you no excuse me i wanna kiss you but not just yet what do you mean not yet only when youre my girl what do you mean your girl my girlfriend miss as if you wish and dont call me miss dont pretend to be a gentleman when youre clearly not so what should i call you rude snob bitch and you douche handsome conceited just like you huh jerk exactly your type leave me alone steph aha steph ill just call you tep remove the s and the f by the way im ton

In [ ]:
df_sample.head()

,num,name,file_content
17262,9251120,maybe this time (2014),watch any video online with opensubtitles free...
7294,9211589,down the shore s01 e10 and justice for all (1992),oh i know that its getting late but i dont wan...
47707,9380845,uncontrollably fond s01 e07 heartache (2016),timing and subtitles by the uncontrollable lov...
29914,9301436,screen two s13 e04 the precious blood (1996),ethereal music apiopensubtitlesorg is deprecat...
54266,9408707,battlebots (2015),chris oh no not the minibots yelling oh you le...


In [ ]:
# Import the required libraries
from sklearn.feature_extraction.text import TfidfVectorizer

# Step 1: Generate BOW / TFIDF sparse vector representations
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(df_sample['file_content'])

In [ ]:
# Display the shape of the TF-IDF matrix
print("TF-IDF matrix shape:", tfidf_vectors.shape)

TF-IDF matrix shape: (24749, 602531)


In [ ]:
! pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 17.8 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Step 2: Generate BERT-based embeddings
sentence_model = SentenceTransformer('bert-base-nli-mean-tokens')
bert_embeddings = np.array([sentence_model.encode(subtitle) for subtitle in df_sample['file_content']])

# Display the shape of the BERT embeddings
# print("BERT embeddings shape:", bert_embeddings.shape)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]